# Importing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os
import json

In [2]:
#Append the parent directory
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

In [3]:
import ps_constructor, likelihood, mcmc_toolkit, data_handling
import logging
from scipy.interpolate import InterpolatedUnivariateSpline
%matplotlib inline
from tqdm import tqdm
from iminuit import Minuit
from dotenv import load_dotenv

# Setting up the environment

In [4]:
#SET UP THE PATHS
MACHINE = 'MAC'
PATHS = data_handling.load_json_to_dict(parent_dir+'/paths.json')[MACHINE]

#ENVIRONMENT NAME
envname = "envs/DATA/Y1/lin/DESI_LRG1_DATA_prerecon.env"

#Load the environment and paths
load_dotenv(os.path.join(PATHS['MAIN_DIR'],envname))
MAIN_DIR = PATHS['MAIN_DIR']
DATA_DIR = PATHS['DATA_DIR']
FIG_DIR = PATHS['FIG_DIR']
CHAIN_DIR = PATHS['CHAIN_DIR']

In [5]:
prior_name = os.getenv('PRIOR_NAME')
prior_file = os.path.join(MAIN_DIR, 'priors', prior_name)

CHAIN_FOLDER = os.getenv('CHAIN_FOLDER')
CHAIN_PATH = os.path.join(CHAIN_DIR, CHAIN_FOLDER, prior_name)

FIG_FOLDER = os.getenv('FIG_FOLDER')
FIG_PATH = os.path.join(FIG_DIR, FIG_FOLDER, prior_name)
DATA_FLAG = True

DATA_NGC_file = os.getenv('DATA_NGC')
DATA_NGC_file = os.path.join(DATA_DIR, DATA_NGC_file)

DATA_SGC_file = os.getenv('DATA_SGC')
DATA_SGC_file = os.path.join(DATA_DIR, DATA_SGC_file)

COV_NGC_file = os.getenv('COV_NGC')
COV_NGC_file = os.path.join(DATA_DIR, COV_NGC_file)

COV_SGC_file = os.getenv('COV_SGC')
COV_SGC_file = os.path.join(DATA_DIR, COV_SGC_file)

fn_wf_ngc = os.getenv('FN_WF_NGC')
if fn_wf_ngc is not None:
    fn_wf_ngc = os.path.join(DATA_DIR, fn_wf_ngc)

fn_wf_sgc = os.getenv('FN_WF_SGC')
if fn_wf_sgc is not None:
    fn_wf_sgc = os.path.join(DATA_DIR, fn_wf_sgc)

In [6]:
# Linear matter power spectrum (smooth and wiggly part)
PLIN = os.getenv('PLIN')
PLIN = os.path.join(MAIN_DIR, PLIN)

# Specify the primordial feature model
primordialfeature_model = os.getenv('MODEL')

# Number of walkers per free parameter
nwalkers_per_param = int(os.getenv('NWALKERS_PER_PARAM'))
initialize_walkers = os.getenv('INITIALIZE_WALKERS')

#Get the mask for the k-range
KMIN = float(os.getenv('KMIN')) if os.getenv('KMIN') is not None else None
KMAX = float(os.getenv('KMAX')) if os.getenv('KMAX') is not None else None

In [7]:
# Create the name of the data file
data_label = envname.split('/')[-1].split('.')[0]

common_name = f"DATA_{data_label}_{prior_name}"

handle = common_name

# Loading the data

In [8]:
# Load the k-array and apply the mask
data_processor = data_handling.DataProcessor(KMIN, KMAX)
k,DATA_NGC = data_processor.load_data(DATA_NGC_file)
k,DATA_SGC = data_processor.load_data(DATA_SGC_file)
DATA = np.concatenate((DATA_NGC, DATA_SGC))

COV_NGC = data_processor.load_cov(COV_NGC_file)
COV_SGC = data_processor.load_cov(COV_SGC_file)
COV = np.block([[COV_NGC, np.zeros_like(COV_NGC)], [np.zeros_like(COV_SGC), COV_SGC]])
invCOV = np.linalg.inv(COV)

#HARTLAP CORRECTION
Nmocks = 1000
Nb = len(k)
invCOV *= (Nmocks-Nb-2-1)/(Nmocks-1)

# Load the window functions
if fn_wf_ngc is not None:
    wfunc_NGC = data_handling.load_winfunc(fn_wf_ngc)
    #Make sure the window function is normalised
    wfunc_NGC[1] = wfunc_NGC[1]/wfunc_NGC[1][0]

if fn_wf_sgc is not None:
    wfunc_SGC = data_handling.load_winfunc(fn_wf_sgc)
    #Make sure the window function is normalised
    wfunc_SGC[1] = wfunc_SGC[1]/wfunc_SGC[1][0]

# Create the log file
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Log the variables
logger.info(f'DATA NGC file: {DATA_NGC_file}')
logger.info(f'DATA SGC file: {DATA_SGC_file}')
logger.info(f'COV NGC file: {COV_NGC_file}')
logger.info(f'Window function (NGC): {fn_wf_ngc}')
logger.info(f'Window function (SGC): {fn_wf_sgc}')
logger.info(f'linear matter power spectrum: {PLIN}')
logger.info(f'primordial feature model: {primordialfeature_model}')
logger.info(f'prior_name: {prior_name}')
logger.info(f'nwalkers_per_param: {nwalkers_per_param}')
logger.info(f'KMIN: {KMIN}')
logger.info(f'KMAX: {KMAX}')
logger.info(f'Filename: {common_name}')

2024-12-12 11:42:35,280 - __main__ - INFO - DATA NGC file: /Users/s2223060/Desktop/LSS_DATA/DATA/LRG/z_0.4_0.6/pre_recon/pkpoles_LRG_NGC_z0.4-0.6_default_FKP_lin_nran18_cellsize6_boxsize7000_d0.001.txt
2024-12-12 11:42:35,283 - __main__ - INFO - DATA SGC file: /Users/s2223060/Desktop/LSS_DATA/DATA/LRG/z_0.4_0.6/pre_recon/pkpoles_LRG_SGC_z0.4-0.6_default_FKP_lin_nran18_cellsize6_boxsize7000_d0.001.txt
2024-12-12 11:42:35,286 - __main__ - INFO - COV NGC file: /Users/s2223060/Desktop/LSS_DATA/EZMocks/LRG/z_0.4_0.6/pre_recon/COV_desi_survey_catalogs_Y1_mocks_SecondGenMocks_EZmock_desipipe_v1_ffa_2pt_pk_pkpoles_LRG_NGC_z0.4-0.6_default_FKP_lin_nran8_cellsize6_boxsize7000_d0.001.txt
2024-12-12 11:42:35,290 - __main__ - INFO - Window function (NGC): /Users/s2223060/Desktop/LSS_DATA/wf/WINDOW_Y1_LRG_NGC_z1.txt
2024-12-12 11:42:35,298 - __main__ - INFO - Window function (SGC): /Users/s2223060/Desktop/LSS_DATA/wf/WINDOW_Y1_LRG_SGC_z1.txt
2024-12-12 11:42:35,301 - __main__ - INFO - linear matter 

# Defining the theory

In [41]:
# Initialize the model for NGC
ps_model_NGC = ps_constructor.PowerSpectrumConstructor(PLIN, primordialfeature_model, k)

# Initialize the model for SGC
ps_model_SGC = ps_constructor.PowerSpectrumConstructor(PLIN, primordialfeature_model, k)

if (fn_wf_ngc is None) or (fn_wf_sgc is None): #No window function convolution
    theory_NGC = lambda x: ps_model_NGC.Evaluate_bare(x)
    theory_SGC = lambda x: ps_model_SGC.Evaluate_bare(x)

else: #Convolve the theory with the window function
    ps_model_NGC.DefineWindowFunction(InterpolatedUnivariateSpline(wfunc_NGC[0],wfunc_NGC[1],ext=3))
    theory_NGC = lambda x: ps_model_NGC.Evaluate_wincov(x)

    ps_model_SGC.DefineWindowFunction(InterpolatedUnivariateSpline(wfunc_SGC[0],wfunc_SGC[1],ext=3))
    theory_SGC = lambda x: ps_model_SGC.Evaluate_wincov(x)

def theory(*theta):
    # Slice theta to get the corresponding values for NGC and SGC
    theta=theta[0]
    
    theta_NGC = theta[0:ndim_NGC]
    theta_SGC = theta[ndim_NGC:ndim_NGC+ndim_SGC]
    shared_params = theta[ndim_NGC+ndim_SGC:]
    
    theta_NGC = np.concatenate([theta_NGC, shared_params])
    theta_SGC = np.concatenate([theta_SGC, shared_params])
    
    # Use np.concatenate to combine the results from both theories
    return np.concatenate((theory_NGC(theta_NGC), theory_SGC(theta_SGC)))

# Initialising the Likelihood

In [42]:
# Load the gelman rubin convergence criteria
with open(os.path.join(MAIN_DIR,'gelman_rubin.json'), 'r') as json_file:
            gelman_rubin = json.load(json_file)

# Initialize the MCMC
mcmc = mcmc_toolkit.MCMC(1, prior_file)
ndim_NGC = len(mcmc.input_prior['NGC'])
ndim_SGC = len(mcmc.input_prior['SGC'])
mcmc.gelman_rubin(gelman_rubin)
mcmc.set_walkers(1)

PrimordialFeature_likelihood = likelihood.likelihoods(theory, DATA, invCOV)

def chi2(*theta):
    return PrimordialFeature_likelihood.chi2(list(theta))

2024-12-12 11:45:21,427 - mcmc_toolkit - INFO - Using /Users/s2223060/Desktop/primordial_features/priors/lin_singlepol file
2024-12-12 11:45:21,429 - mcmc_toolkit - INFO - Input dictionary: OrderedDict([('NGC', OrderedDict([('BNGC', [0.1, 8])])), ('SGC', OrderedDict([('BSGC', [0.1, 8])])), ('a0', [-1, 1]), ('a1', [-10, 10]), ('a2', [-50, 50]), ('a3', [-50, 50]), ('a4', [-200, 200]), ('alpha', [0.8, 1.2]), ('sigma_nl', [0, 15]), ('sigma_s', [0, 15]), ('A', [-5, 5]), ('omega', [0, 0]), ('phi', [0, 0.5])])
2024-12-12 11:45:21,430 - mcmc_toolkit - INFO - Expanded dictionary: OrderedDict([('BNGC', [0.1, 8]), ('BSGC', [0.1, 8]), ('a0', [-1, 1]), ('a1', [-10, 10]), ('a2', [-50, 50]), ('a3', [-50, 50]), ('a4', [-200, 200]), ('alpha', [0.8, 1.2]), ('sigma_nl', [0, 15]), ('sigma_s', [0, 15]), ('A', [-5, 5]), ('omega', [0, 0]), ('phi', [0, 0.5])])
2024-12-12 11:45:21,431 - mcmc_toolkit - INFO - Parameter labels: ['BNGC', 'BSGC', 'a0', 'a1', 'a2', 'a3', 'a4', 'alpha', 'sigma_nl', 'sigma_s', 'A', '

# MINUIT

In [67]:
omegas = np.arange(105,4005,10)
limits = [(x[0],x[1]) for x in mcmc.prior_bounds.T]

X0_str = os.getenv("X0")
DELTA_str = os.getenv('DELTA')

if X0_str:
    X0 = np.array([float(x) for x in X0_str.split(',')])
    DELTA = np.array([float(x) for x in DELTA_str.split(',')])
else:
    X0 = np.array([])  # or handle the case where X0 is not set
    DELTA = np.array([])
    logger.warning('X0 and SIGMA not set')

In [68]:
params = []
chi2_list = []

In [69]:
for i in range(0,10):
    print(i)
    limits[mcmc.id_map['omega']] = (omegas[i],omegas[i])
    initial_positions = mcmc.create_walkers(initialize_walkers,x0 =X0,delta = DELTA)[0]
    initial_positions[mcmc.id_map['omega']] = omegas[i]
    
    m = Minuit(chi2, name = mcmc.labels, **{x:val for x,val in zip(mcmc.labels, initial_positions)})
    m.limits = limits
    m.migrad(ncall = 20000)
    params.append(list(m.values.to_dict().values()))
    chi2_list.append(m.fmin.fval)

2024-12-12 11:55:55,537 - mcmc_toolkit - INFO - Using the uniform_thin walker positioning


0


2024-12-12 11:56:12,416 - mcmc_toolkit - INFO - Using the uniform_thin walker positioning


1


2024-12-12 11:56:36,145 - mcmc_toolkit - INFO - Using the uniform_thin walker positioning


...

7


2024-12-12 11:58:52,097 - mcmc_toolkit - INFO - Using the uniform_thin walker positioning


8


2024-12-12 11:59:12,208 - mcmc_toolkit - INFO - Using the uniform_thin walker positioning


9


In [ ]:
 m.values,  # Parameter names and their fitted values
 m.errors,  # Parameter names and their HESSE errors
 {str(k): v for k, v in m.covariance.items()},  # Covariance matrix as serializable dict
 m.covariance.correlation().tolist(),  # Correlation matrix as a list of lists
 m.fmin._asdict(),  # Function minimum details as a dictionary
 m.nfcn,  # Number of function evaluations
 m.edm,  # Estimated distance to the minimum

In [ ]:
x0 = list(m.values.to_dict().values())

In [ ]:
theory_bf = theory(x0)
plt.figure()
plt.plot(k, k*theory_bf[0:len(k)])
plt.plot(k, k*theory_bf[len(k):])
plt.plot(k, k*DATA[0:len(k)])
plt.plot(k, k*DATA[len(k):])

In [ ]:
x = m.values.to_dict()

In [ ]:
bf_NGC = theory_NGC([x['BNGC'],x['a0NGC'],x['a1NGC'],x['a2NGC'],x['a3NGC'],x['a4NGC'], x['alpha'],x['sigma_nl'],x['sigma_s']])
bf_SGC = theory_SGC([x['BSGC'],x['a0SGC'],x['a1SGC'],x['a2SGC'],x['a3SGC'],x['a4SGC'], x['alpha'],x['sigma_nl'],x['sigma_s']])

In [ ]:
bf_smooth_NGC = theory_NGC([x['BNGC'],x['a0NGC'],x['a1NGC'],x['a2NGC'],x['a3NGC'],x['a4NGC'],x['alpha'],10000,x['sigma_s']])
bf_smooth_SGC = theory_SGC([x['BSGC'],x['a0SGC'],x['a1SGC'],x['a2SGC'],x['a3SGC'],x['a4SGC'],x['alpha'],10000,x['sigma_s']])

In [ ]:
sigma_NGC = np.diag(np.linalg.inv(invcov))[0:len(k)]**0.5
sigma_SGC = np.diag(np.linalg.inv(invcov))[len(k):]**0.5

In [ ]:
DATA_NGC,DATA_SGC = DATA[0:len(k)],  DATA[len(k):]
fig, ax = plt.subplots(1,2, figsize = (8,4))
ax[0].plot(k, (DATA_NGC/bf_smooth_NGC))
ax[0].plot(k, (bf_NGC/bf_smooth_NGC), color = 'black', lw = 4)
ax[0].set_xlim((k[0],k[-1]))
ax[0].set_ylim((0.8,1.2))
ax[0].grid()
ax[0].set_title(tracer)
ax[1].plot(k, (DATA_SGC/bf_smooth_SGC))
ax[1].plot(k, (bf_SGC/bf_smooth_SGC), color = 'black', lw = 4)
ax[1].set_xlim((k[0],k[-1]))
ax[1].set_ylim((0.8,1.2))
ax[1].grid()
ax[1].set_title(r"$\chi^2/\mathrm{dof} = $"+f" {m.fval:.02f}/{2*len(k)}-{len(x)}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Split data
DATA_NGC, DATA_SGC = DATA[0:len(k)], DATA[len(k):]

# Create figure and grid layout
fig = plt.figure(figsize=(10, 8))
gs = gridspec.GridSpec(2, 2, height_ratios=[3, 1], hspace=0.05)

# Main plot for NGC
ax_main_ngc = fig.add_subplot(gs[0, 0])
ax_main_ngc.plot(k, (DATA_NGC / bf_smooth_NGC), marker='o', label="Data / Model")
ax_main_ngc.plot(k, (bf_NGC / bf_smooth_NGC), color='black', lw=4, label="Best Fit / Model")
ax_main_ngc.set_xlim((k[0], k[-1]))
ax_main_ngc.set_ylim((0.8, 1.2))
ax_main_ngc.grid()
ax_main_ngc.set_title(tracer)
ax_main_ngc.legend()

# Residual plot for NGC
ax_res_ngc = fig.add_subplot(gs[1, 0], sharex=ax_main_ngc)
residuals_ngc = (bf_NGC - DATA_NGC) / sigma_NGC
ax_res_ngc.scatter(k, residuals_ngc, color='blue', label="Residuals", s = 1,marker = "s")
ax_res_ngc.axhline(0, color='black', lw=1, linestyle='--')
ax_res_ngc.set_xlim((k[0], k[-1]))
ax_res_ngc.set_ylim((-5, 5))
ax_res_ngc.set_ylabel("Residuals")
ax_res_ngc.grid()

# Main plot for SGC
ax_main_sgc = fig.add_subplot(gs[0, 1])
ax_main_sgc.plot(k, (DATA_SGC / bf_smooth_SGC), marker='o', label="Data / Model")
ax_main_sgc.plot(k, (bf_SGC / bf_smooth_SGC), color='black', lw=4, label="Best Fit / Model")
ax_main_sgc.set_xlim((k[0], k[-1]))
ax_main_sgc.set_ylim((0.8, 1.2))
ax_main_sgc.grid()
ax_main_sgc.set_title(r"$\chi^2/\mathrm{dof} = $" + f" {m.fval:.02f}/{2*len(k)}-{len(x)}")
ax_main_sgc.legend()

# Residual plot for SGC
ax_res_sgc = fig.add_subplot(gs[1, 1], sharex=ax_main_sgc)
residuals_sgc = (bf_SGC - DATA_SGC) / sigma_SGC
ax_res_sgc.scatter(k, residuals_sgc, color='blue', label="Residuals", s = 1,marker  = "s")
ax_res_sgc.axhline(0, color='black', lw=1, linestyle='--')
ax_res_sgc.set_xlim((k[0], k[-1]))
ax_res_sgc.set_ylim((-5, 5))
ax_res_sgc.grid()

# Final adjustments
plt.show()

# Checking results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import json

# Function to load a .json file as a dictionary
def load_json_as_dict(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"Error: File {file_path} not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {file_path}.")
        return None


In [ ]:
results_dir = '/scratch/dp322/dc-merg1/chains/DESI_QSO_MOCK_Abacus/BAO_doublepol/MINUIT_MINUIT_BAO_doublepol_2pt_mock0_recon_sm30_recsym_pkpoles_QSO_default_FKP_lin_nran18_cellsize6_boxsize10000_d0.001_results.json'

In [ ]:
 },
    "LRG2":{
    "prerecon":"DESI_LRG2_MOCK_Abacus/BAO_doublepol/MINUIT_MINUIT_BAO_doublepol_2pt_mock{}_pk_pkpoles_LRG__z0.6-0.8_default_FKP_lin_nran18_cellsize6_boxsize7000_thetacut0.05_d0.001_results.json",
    "recsym":"DESI_LRG2_MOCK_Abacus/BAO_doublepol/MINUIT_MINUIT_BAO_doublepol_2pt_mock{}_recon_sm15_IFFT_recsym_pk_pkpoles_LRG__z0.6-0.8_default_FKP_lin_nran18_cellsize6_boxsize7000_d0.001_results.json"
    },
    "LRG3":{
    "prerecon":"/scratch/dp322/dc-merg1/chains/DESI_LRG3_MOCK_Abacus/BAO_doublepol/MINUIT_BAO_doublepol_2pt_mock{}_pk_pkpoles_LRG__z0.8-1.1_default_FKP_lin_nran18_cellsize6_boxsize7000_thetacut0.05_d0.001_results.json",
    "recsym":"/scratch/dp322/dc-merg1/chains/DESI_LRG3_MOCK_Abacus/BAO_doublepol/MINUIT_BAO_doublepol_2pt_mock{}_recon_sm15_IFFT_recsym_pk_pkpoles_LRG__z0.8-1.1_default_FKP_lin_nran18_cellsize6_boxsize7000_d0.001_results.json"
    },
    "QSO":{
        'prerecon':'/scratch/dp322/dc-merg1/chains/DESI_QSO_MOCK_Abacus/BAO_doublepol/MINUIT_BAO_doublepol_2pt_mock{}_pk_pkpoles_QSO_default_FKP_lin_nran18_cellsize6_boxsize10000_d0.001_results.json',
        'recsym':'/scratch/dp322/dc-merg1/chains/DESI_QSO_MOCK_Abacus/BAO_doublepol/MINUIT_BAO_doublepol_2pt_mock{}_recon_sm30_recsym_pkpoles_QSO_default_FKP_lin_nran18_cellsize6_boxsize10000_d0.001_results.json'
    }

In [ ]:
results_fn = {
    "LRG1":{
        "prerecon":"/scratch/dp322/dc-merg1/chains/DESI_LRG1_MOCK_Abacus/BAO_doublepol/MINUIT_BAO_doublepol_2pt_mock{}_pk_pkpoles_LRG__z0.4-0.6_default_FKP_lin_nran18_cellsize6_boxsize7000_thetacut0.05_d0.001_results.json",
        "recsym":"/scratch/dp322/dc-merg1/chains/DESI_LRG1_MOCK_Abacus/BAO_doublepol/MINUIT_BAO_doublepol_2pt_mock{}_recon_sm15_IFFT_recsym_pk_pkpoles_LRG__z0.4-0.6_default_FKP_lin_nran18_cellsize6_boxsize7000_d0.001_results.json"},
    "LRG2":{
    "prerecon":"/scratch/dp322/dc-merg1/chains/DESI_LRG2_MOCK_Abacus/BAO_doublepol/MINUIT_BAO_doublepol_2pt_mock{}_pk_pkpoles_LRG__z0.6-0.8_default_FKP_lin_nran18_cellsize6_boxsize7000_thetacut0.05_d0.001_results.json",
    "recsym":"/scratch/dp322/dc-merg1/chains/DESI_LRG2_MOCK_Abacus/BAO_doublepol/MINUIT_BAO_doublepol_2pt_mock{}_recon_sm15_IFFT_recsym_pk_pkpoles_LRG__z0.6-0.8_default_FKP_lin_nran18_cellsize6_boxsize7000_d0.001_results.json"
    },
    "LRG3":{
    "prerecon":"/scratch/dp322/dc-merg1/chains/DESI_LRG3_MOCK_Abacus/BAO_doublepol/MINUIT_BAO_doublepol_2pt_mock{}_pk_pkpoles_LRG__z0.8-1.1_default_FKP_lin_nran18_cellsize6_boxsize7000_thetacut0.05_d0.001_results.json",
    "recsym":"/scratch/dp322/dc-merg1/chains/DESI_LRG3_MOCK_Abacus/BAO_doublepol/MINUIT_BAO_doublepol_2pt_mock{}_recon_sm15_IFFT_recsym_pk_pkpoles_LRG__z0.8-1.1_default_FKP_lin_nran18_cellsize6_boxsize7000_d0.001_results.json"
    },
    "QSO":{
        'prerecon':'/scratch/dp322/dc-merg1/chains/DESI_QSO_MOCK_Abacus/BAO_doublepol/MINUIT_BAO_doublepol_2pt_mock{}_pk_pkpoles_QSO_default_FKP_lin_nran18_cellsize6_boxsize10000_d0.001_results.json',
        'recsym':'/scratch/dp322/dc-merg1/chains/DESI_QSO_MOCK_Abacus/BAO_doublepol/MINUIT_BAO_doublepol_2pt_mock{}_recon_sm30_recsym_pkpoles_QSO_default_FKP_lin_nran18_cellsize6_boxsize10000_d0.001_results.json'
    }
}

results = {}
for tracer, all_results in results_fn.items():
    results[tracer] = {}
    for recon_status,fn in results_fn[tracer].items():
        results[tracer][recon_status] = {}
        alpha = []
        sigma_alpha = []
        chi2 = []
        fn = results_fn[tracer][recon_status]
        print(fn)
        for i in range(1,25):
            data = load_json_as_dict(fn.format(i))
            alpha.append(data["param_values"][12])
            sigma_alpha.append(data["param_errors"]['alpha'])
            chi2.append(data['fmin'])
        results[tracer][recon_status]['alpha'] = alpha
        results[tracer][recon_status]['sigma_alpha'] = sigma_alpha
        results[tracer][recon_status]['chi2'] = chi2

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Create the figure and GridSpec
fig = plt.figure(figsize=(12, 6))
gs = GridSpec(2, 3, figure=fig, hspace=0.3, wspace=0.3)

# Create axes using GridSpec
ax = [
    [fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1]), fig.add_subplot(gs[0, 2])],
    [fig.add_subplot(gs[1, 0]), fig.add_subplot(gs[1, 1]), fig.add_subplot(gs[1, 2])],]

# Define the data keys and row indices for prerecon and recsym
data_keys = ['chi2', 'alpha', 'sigma_alpha']
row_data = {
    0: results['LRG1']['prerecon'],  # First row: prerecon
    1: results['LRG1']['recsym']     # Second row: recsym
}

# Plotting histograms
for row, data in row_data.items():
    for col, key in enumerate(data_keys):
        if col == 0:
            ax[row][col].hist(np.array(data[key])/545, color='green', bins = 10)
        else:
            ax[row][col].hist(data[key], color='green', bins = 10)

# Adding grid and sharing x-axis
for i in range(2):
    for j in range(3):
        #ax[i][j].grid(True)
        if i == 1:  # Share x-axis between rows
            ax[i][j].sharex(ax[0][j])
        if j == 0:
            ax[i][j].set_xlabel(r"$\chi^2$")
        if j == 1:
            ax[i][j].set_xlabel(r"$\alpha$")
        if j == 2:
            ax[i][j].set_xlabel(r"$\sigma(\alpha)$")
            
# Adding annotations
ax[0][0].set_title('LRG1')
ax[0][0].annotate('Prerecon', xy=(0.2, 0.9), xycoords='axes fraction', ha='center', fontsize=12)
ax[1][0].annotate('Recsym', xy=(0.2, 0.9), xycoords='axes fraction', ha='center', fontsize=12)

# Adjust layout
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Create the figure and GridSpec
fig = plt.figure(figsize=(12, 6))
gs = GridSpec(2, 3, figure=fig, hspace=0.3, wspace=0.3)

# Create axes using GridSpec
ax = [
    [fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1]), fig.add_subplot(gs[0, 2])],
    [fig.add_subplot(gs[1, 0]), fig.add_subplot(gs[1, 1]), fig.add_subplot(gs[1, 2])],]

# Define the data keys and row indices for prerecon and recsym
data_keys = ['chi2', 'alpha', 'sigma_alpha']
row_data = {
    0: results['LRG2']['prerecon'],  # First row: prerecon
    1: results['LRG2']['recsym']     # Second row: recsym
}

# Plotting histograms
for row, data in row_data.items():
    for col, key in enumerate(data_keys):
        if col == 0:
            ax[row][col].hist(np.array(data[key])/545, color='blue', bins = 10)
        else:
            ax[row][col].hist(data[key], color='blue', bins = 10)

# Adding grid and sharing x-axis
for i in range(2):
    for j in range(3):
        #ax[i][j].grid(True)
        if i == 1:  # Share x-axis between rows
            ax[i][j].sharex(ax[0][j])
        if j == 0:
            ax[i][j].set_xlabel(r"$\chi^2$")
        if j == 1:
            ax[i][j].set_xlabel(r"$\alpha$")
        if j == 2:
            ax[i][j].set_xlabel(r"$\sigma(\alpha)$")
            
# Adding annotations
ax[0][0].set_title('LRG2')
ax[0][0].annotate('Prerecon', xy=(0.2, 0.9), xycoords='axes fraction', ha='center', fontsize=12)
ax[1][0].annotate('Recsym', xy=(0.2, 0.9), xycoords='axes fraction', ha='center', fontsize=12)

# Adjust layout
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Create the figure and GridSpec
fig = plt.figure(figsize=(12, 6))
gs = GridSpec(2, 3, figure=fig, hspace=0.3, wspace=0.3)

# Create axes using GridSpec
ax = [
    [fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1]), fig.add_subplot(gs[0, 2])],
    [fig.add_subplot(gs[1, 0]), fig.add_subplot(gs[1, 1]), fig.add_subplot(gs[1, 2])],]

# Define the data keys and row indices for prerecon and recsym
data_keys = ['chi2', 'alpha', 'sigma_alpha']
row_data = {
    0: results['LRG3']['prerecon'],  # First row: prerecon
    1: results['LRG3']['recsym']     # Second row: recsym
}

# Plotting histograms
for row, data in row_data.items():
    for col, key in enumerate(data_keys):
        if col == 0:
            ax[row][col].hist(np.array(data[key])/545, color='red', bins = 10)
        else:
            ax[row][col].hist(data[key], color='red', bins = 10)

# Adding grid and sharing x-axis
for i in range(2):
    for j in range(3):
        #ax[i][j].grid(True)
        if i == 1:  # Share x-axis between rows
            ax[i][j].sharex(ax[0][j])
        if j == 0:
            ax[i][j].set_xlabel(r"$\chi^2$")
        if j == 1:
            ax[i][j].set_xlabel(r"$\alpha$")
        if j == 2:
            ax[i][j].set_xlabel(r"$\sigma(\alpha)$")
            
# Adding annotations
ax[0][0].set_title('LRG3')
ax[0][0].annotate('Prerecon', xy=(0.2, 0.9), xycoords='axes fraction', ha='center', fontsize=12)
ax[1][0].annotate('Recsym', xy=(0.2, 0.9), xycoords='axes fraction', ha='center', fontsize=12)
#ax[1][2].vlines(0.00249*5,0,15,color = "black",lw = 3,ls='--')
# Adjust layout
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Create the figure and GridSpec
fig = plt.figure(figsize=(12, 6))
gs = GridSpec(2, 3, figure=fig, hspace=0.3, wspace=0.3)

# Create axes using GridSpec
ax = [
    [fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1]), fig.add_subplot(gs[0, 2])],
    [fig.add_subplot(gs[1, 0]), fig.add_subplot(gs[1, 1]), fig.add_subplot(gs[1, 2])],]

# Define the data keys and row indices for prerecon and recsym
data_keys = ['chi2', 'alpha', 'sigma_alpha']
row_data = {
    0: results['QSO']['prerecon'],  # First row: prerecon
    1: results['QSO']['recsym']     # Second row: recsym
}

# Plotting histograms
for row, data in row_data.items():
    for col, key in enumerate(data_keys):
        if col == 0:
            ax[row][col].hist(np.array(data[key])/545, color='purple', bins = 10)
        else:
            ax[row][col].hist(data[key], color='purple', bins = 10)

# Adding grid and sharing x-axis
for i in range(2):
    for j in range(3):
        #ax[i][j].grid(True)
        if i == 1:  # Share x-axis between rows
            ax[i][j].sharex(ax[0][j])
        if j == 0:
            ax[i][j].set_xlabel(r"$\chi^2$")
        if j == 1:
            ax[i][j].set_xlabel(r"$\alpha$")
        if j == 2:
            ax[i][j].set_xlabel(r"$\sigma(\alpha)$")
            
# Adding annotations
ax[0][0].set_title('QSO')
ax[0][0].annotate('Prerecon', xy=(0.2, 0.9), xycoords='axes fraction', ha='center', fontsize=12)
ax[1][0].annotate('Recsym', xy=(0.2, 0.9), xycoords='axes fraction', ha='center', fontsize=12)
#ax[1][2].vlines(0.0049*5,0,15,color = "black",lw = 3,ls='--')
#ax[1][2].set_ylim((0,10))
# Adjust layout
plt.show()

In [ ]:
for t in ['LRG1','LRG2','LRG3','QSO']:
    for r in ['prerecon','recsym']:
        print(t,r)
        print(np.mean(results[t][r]['alpha']), "+-", np.std(results[t][r]['alpha']))